In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dir_data = './data/'

f_app = os.path.join(dir_data, 'application_train.csv')
print('Path of read in data: %s' % (f_app))
app_train = pd.read_csv(f_app)
app_train.head()

description = os.path.join(dir_data, 'HomeCredit_columns_description.csv')
description = pd.read_csv(description, encoding = 'latin1')

dtype_select = [np.dtype('float64'), np.dtype('int64')]

# list(app_train.dtypes.isin(dtype_select)) 為一個內容為 True or False 的 list
# app_train.columns[filter_list()]) 會傳回 columns，其中 filter_list's element = True
# 所以這裡會回傳 dtype = int64 或 float 64 的 columns
numeric_columns = list(app_train.columns[list(app_train.dtypes.isin(dtype_select))])

# 再把只有 2 值 (通常是 0,1) 的欄位去掉
numeric_columns = list(app_train[numeric_columns].columns[list(app_train[numeric_columns].apply(lambda x:len(x.unique())!=2 ))])

# 檢視這些欄位的數值範圍
#for col in numeric_columns:
#     # 顯示所有的欄位
#     plt.boxplot(app_train[col], vert=False)
#     plt.title(col)
#     plt.show()
    # 只顯示最大值-平均值，遠大於標準差的欄位
    #if ((app_train[col].max() - app_train[col].mean())/app_train[col].std() > 100):
    #    app_train.boxplot(column=col, vert=False)
    #    print("mean = %f, max = %f" % (app_train[col].mean(), app_train[col].max()))
    #    plt.title(col)
    #    plt.show()


# 觀察以上列出三項的統計資料
items = ['AMT_INCOME_TOTAL', 'REGION_POPULATION_RELATIVE', 'OBS_60_CNT_SOCIAL_CIRCLE']
#print(app_train[items].describe())

# 最大值離平均與中位數很遠
print(app_train['AMT_INCOME_TOTAL'].describe())

# 繪製 Empirical Cumulative Density Plot (ECDF)
cdf = app_train['AMT_INCOME_TOTAL']
# print(cdf.shape, cdf.min(), cdf.max())   # (307511,) 25650.0 117000000.0
# X軸: cdf.min() ~ cdf.max()
# Y軸: 累進人數/總人數

# value_counts(): 對 value 做記數，算出每個 value 有多少個數
# sort_index(): 再對 index 做排序，應該我們要觀察的是 X軸由小到大
# cumsum(): 做累進加法。也就是 FX(x)的個數
cdf = cdf.value_counts().sort_index().cumsum()
# cdf.tail() 可以看出有那些異常大的值
print(cdf.tail())

plt.plot(list(cdf.index), cdf/cdf.max())
plt.xlabel('Value')
plt.ylabel('ECDF')
plt.xlim([cdf.index.min(), cdf.index.max() * 1.05]) # 限制顯示圖片的範圍
plt.ylim([-0.05,1.05]) # 限制顯示圖片的範圍

plt.show()

# 改變 y 軸的 Scale, 讓我們可以正常檢視 ECDF
plt.plot(np.log(list(cdf.index)), cdf/cdf.max())
plt.xlabel('Value (log-scale)')
plt.ylabel('ECDF')

plt.ylim([-0.05,1.05]) # 限制顯示圖片的範圍

plt.show()

# 最大值落在分布之外
print(app_train['REGION_POPULATION_RELATIVE'].describe())

# 繪製 Empirical Cumulative Density Plot (ECDF)
cdf = app_train['REGION_POPULATION_RELATIVE'].value_counts().sort_index().cumsum()

plt.plot(list(cdf.index), cdf/cdf.max())
plt.xlabel('Value')
plt.ylabel('ECDF')
plt.ylim([-0.05,1.05]) # 限制顯示圖片的範圍
plt.show()

app_train['REGION_POPULATION_RELATIVE'].hist()
plt.show()

app_train['REGION_POPULATION_RELATIVE'].value_counts()

# 就以這個欄位來說，雖然有資料掉在分布以外，也不算異常，僅代表這間公司在稍微熱鬧的地區有的據點較少，
# 導致 region population relative 在少的部分較為密集，但在大的部分較為疏漏

# 最大值落在分布之外
print(app_train['OBS_60_CNT_SOCIAL_CIRCLE'].describe())

# 繪製 Empirical Cumulative Density Plot (ECDF)
cdf = app_train['OBS_60_CNT_SOCIAL_CIRCLE'].value_counts().sort_index().cumsum()
plt.plot(list(cdf.index), cdf/cdf.max())
plt.xlabel('Value')
plt.ylabel('ECDF')
plt.xlim([cdf.index.min() * 0.95, cdf.index.max() * 1.05])
plt.ylim([-0.05,1.05]) # 限制顯示圖片的範圍
plt.show()

app_train['OBS_60_CNT_SOCIAL_CIRCLE'].hist()
plt.show()
print(app_train['OBS_60_CNT_SOCIAL_CIRCLE'].value_counts().sort_index(ascending = False))

# 把一些極端值暫時去掉，在繪製一次 Histogram
# 選擇 OBS_60_CNT_SOCIAL_CIRCLE 小於 20 的資料點繪製
loc_a = app_train['OBS_60_CNT_SOCIAL_CIRCLE'] < 20
loc_b = 'OBS_60_CNT_SOCIAL_CIRCLE'

app_train.loc[loc_a, loc_b].hist()
plt.show()